In [23]:
import json
import pandas as pd

FOLDER = "output"

file = "Xp8pH3tCQ5E.json"

## TO-DO: Use Path instead

INPUT_FILE = f"{FOLDER}/{file}"

df = pd.read_json(INPUT_FILE)

# explode dicts

df = df.join(pd.json_normalize(df["content"])).drop(columns=["content"])

In [24]:
df.columns

Index(['message_type', 'occurence_timestamp', 'timeStampUTC', 'author',
       'message', 'emoji', 'text_content', 'purchaseAmount.simpleText',
       'giftText', 'giftedText', 'membershipChat', 'membershipJoin',
       'membershipRenewal'],
      dtype='object')

In [25]:
df.author.nunique()

1401

In [26]:
df.author.value_counts()

author
Murilo Torres♰         294
Ilana                  147
N/A                    147
Mateus Matias          122
Sandro                 106
                      ... 
BRENO DC YT              1
NANDOLLA TV              1
Gab                      1
Ticquey HUE              1
Marcelo Rocha Brito      1
Name: count, Length: 1401, dtype: int64

In [27]:
df.message_type.value_counts()

message_type
Chat Message                        6905
Placeholder Item Message             147
Superchat Message                     95
Membership Gift Received message      17
Membership Chat message               10
Membership Gift message                3
Membership Join message                2
Membership Renew message               2
Name: count, dtype: int64

In [28]:
df["purchaseAmount.simpleText"].value_counts()

purchaseAmount.simpleText
R$2.00     60
R$5.00     26
R$10.90     3
R$10.00     2
R$1.00      2
$5.00       1
R$2.01      1
Name: count, dtype: int64

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7181 entries, 0 to 7180
Data columns (total 13 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   message_type               7181 non-null   object        
 1   occurence_timestamp        7178 non-null   object        
 2   timeStampUTC               7181 non-null   datetime64[ns]
 3   author                     7181 non-null   object        
 4   message                    7147 non-null   object        
 5   emoji                      6905 non-null   object        
 6   text_content               6905 non-null   object        
 7   purchaseAmount.simpleText  95 non-null     object        
 8   giftText                   3 non-null      object        
 9   giftedText                 17 non-null     object        
 10  membershipChat             10 non-null     object        
 11  membershipJoin             2 non-null      object        
 12  member

In [30]:
df["purchaseAmount.simpleText"].fillna("R$ 0.00", inplace=True)

/tmp/ipykernel_258212/653160005.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["purchaseAmount.simpleText"].fillna("R$ 0.00", inplace=True)


In [31]:
import re
def parse_currency_and_value(currency_string):
    # Regular expression to extract the currency symbol and the numeric value
    pattern = re.compile(r'([^\d.,]+)?([\d.,]+)')
    match = pattern.match(currency_string.strip())

    if match:
        currency = match.group(1)
        currency = currency.strip() if currency else ''
        value = float(match.group(2).replace(',', ''))  # Handle cases where commas are used as thousand separators
        return currency, value
    else:
        raise ValueError("Invalid currency format")

    
df["currency"], df["amount"] = zip(*df["purchaseAmount.simpleText"].map(parse_currency_and_value))

In [32]:
df["amount"].value_counts()

amount
0.00     7086
2.00       60
5.00       27
10.90       3
10.00       2
1.00        2
2.01        1
Name: count, dtype: int64

In [33]:
df["amount"].sum().round(2)

np.float64(311.71)